## Without the package

In [27]:
import numpy as np

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
valid_size = 0.2
batch_size = 32

transform_training = transforms.Compose(
    [transforms.RandomRotation(15),   
     transforms.RandomResizedCrop(224),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(saturation=0.2, contrast=0.2, hue=0.2, brightness=0.2),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)]
)
transform_testing = transforms.Compose(
    [transforms.Resize((224,224)),
    transforms.ToTensor(), 
    transforms.Normalize(mean, std)]
)

train_data = datasets.ImageFolder("./raw_images_clean/train", transform=transform_training)
valid_data = datasets.ImageFolder("./raw_images_clean/valid", transform=transform_training)
test_data = datasets.ImageFolder("./raw_images_clean/test", transform=transform_testing)

# And now we create the loders
train_loader = DataLoader(train_data, batch_size=batch_size)
valid_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)


loaders_scratch = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

print(len(train_loader), len(valid_loader), len(test_loader))

395 395 16


In [32]:
# useful variable that tells us whether we should use the GPU
import torch
use_cuda = torch.cuda.is_available()

In [31]:
import torch.nn as nn
from torch import optim
criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    return optim.Adam(model.parameters(), lr=0.0003)

In [34]:
import torch.nn as nn
import torch.nn.functional as F

number_classes = len(train_data.classes)

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv_1 = nn.Conv2d(3, 16, 3, stride=1, padding=1) 
        self.conv_2 = nn.Conv2d(16, 32, 3, stride=1, padding=1) 
        self.conv_3 = nn.Conv2d(32, 64, 3, stride=1, padding=1) 
        self.conv_4 = nn.Conv2d(64, 128, 3, stride=1, padding=1) 
        self.conv_5 = nn.Conv2d(128, 256, 3, stride=1, padding=1) 

        
        # And also define a max pooling layer
        self.pool = nn.MaxPool2d(2,2)        
        
        # And linear models
        self.fc1 = nn.Linear(7*7*256, 1024) # 7 as (224/(2*2*2*2*2) and 256 as it is the depth

        # Other model that i tested
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, number_classes)

        
        # Will define a dropout of 0.3
        self.dropout = nn.Dropout(0.2)
        
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv_1(x)))
        x = self.pool(F.relu(self.conv_2(x)))
        x = self.pool(F.relu(self.conv_3(x)))
        x = self.pool(F.relu(self.conv_4(x)))
        x = self.pool(F.relu(self.conv_5(x)))

        
        # Flatten the input
        x = x.view(-1, 7*7*256)
        
        # Add the dropout
        x = self.dropout(x)
        
        # Add the linear layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

model_scratch

Net(
  (conv_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [38]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            # Obtain the output from the model
            output = model(data)
            # Obtain loss
            loss = criterion(output, target)
            # Backward induction
            loss.backward()
            # Perform optimization step
            optimizer.step()            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data)
            # Obtain the loss
            loss = criterion(output, target)
            # Add this loss to the list (same as before but instead of train we use valid)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))
            

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
    
        if valid_loss < valid_loss_min:
            # Print an alert
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model..'.format(
                valid_loss_min,
                valid_loss))

            torch.save(model.state_dict(), save_path)
            
            # Update the new minimum
            valid_loss_min = valid_loss
        
    return model

In [39]:
def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    classname = m.__class__.__name__
    
    # for every Linear layer in a model >
    if classname.find('Linear') != -1:
        # Get the number of the inputs
        n = m.in_features
        # Define which will be the y
        y = (1.0/np.sqrt(n))
        m.weight.data.normal_(0, y) 
        # Bias will be filled with 0
        m.bias.data.fill_(0)


#-#-# Do NOT modify the code below this line. #-#-#
    
model_scratch.apply(custom_weight_init)
model_scratch = train(
    20, 
    loaders_scratch, 
    model_scratch, 
    get_optimizer_scratch(model_scratch),
    criterion_scratch, 
    use_cuda, 
    'ignore.pt'
)

tensor([[[[ 0.8276,  0.7591,  0.6734,  ..., -0.3027,  0.0912,  0.3652],
          [ 0.9132,  0.8447,  0.7933,  ..., -0.4911, -0.0458,  0.2967],
          [ 0.9817,  0.9646,  0.9303,  ..., -0.8164, -0.2513,  0.1426],
          ...,
          [-0.1999, -0.1657, -0.1486,  ...,  0.7248,  0.6049,  0.5022],
          [-0.0801, -0.1486, -0.2342,  ...,  0.8104,  0.6392,  0.5364],
          [-0.0116, -0.1314, -0.3027,  ...,  0.8447,  0.6734,  0.5536]],

         [[ 0.4853,  0.4678,  0.3978,  ..., -0.3550,  0.0476,  0.3627],
          [ 0.4853,  0.4678,  0.4328,  ..., -0.5826, -0.0749,  0.2752],
          [ 0.4678,  0.4678,  0.4853,  ..., -0.8978, -0.3025,  0.1001],
          ...,
          [-0.4601, -0.4251, -0.3901,  ...,  0.6254,  0.4503,  0.2927],
          [-0.3550, -0.3901, -0.4776,  ...,  0.6779,  0.4853,  0.3452],
          [-0.2675, -0.3725, -0.5651,  ...,  0.7304,  0.5203,  0.3627]],

         [[ 0.0779, -0.0267, -0.1661,  ..., -0.2358,  0.1825,  0.4788],
          [ 0.1476,  0.0953, -

tensor([[[[-0.5938, -0.5424, -0.4739,  ..., -0.6794, -0.5424, -0.4911],
          [-0.5767, -0.5424, -0.4911,  ..., -0.6794, -0.5596, -0.5082],
          [-0.5596, -0.5424, -0.5082,  ..., -0.6965, -0.5938, -0.5767],
          ...,
          [-0.6623, -0.6623, -0.6452,  ..., -1.1418, -1.0562, -1.0219],
          [-0.6965, -0.6794, -0.6452,  ..., -1.2788, -0.9192, -0.7308],
          [-0.7137, -0.6965, -0.6452,  ..., -1.3130, -0.8507, -0.6109]],

         [[-0.5826, -0.5476, -0.4776,  ..., -0.6001, -0.4601, -0.3901],
          [-0.5651, -0.5476, -0.4951,  ..., -0.6001, -0.4776, -0.4251],
          [-0.5651, -0.5476, -0.5301,  ..., -0.6001, -0.5126, -0.4951],
          ...,
          [-0.7402, -0.7402, -0.7402,  ..., -0.9328, -0.8978, -0.8452],
          [-0.7752, -0.7577, -0.7402,  ..., -1.0728, -0.7227, -0.5301],
          [-0.7927, -0.7752, -0.7402,  ..., -1.1253, -0.6527, -0.4251]],

         [[-0.5844, -0.5495, -0.4624,  ..., -0.3055, -0.1661, -0.1138],
          [-0.5844, -0.5495, -

tensor([[[[ 1.8550,  1.8550,  1.8550,  ...,  1.8379,  1.7865,  1.7180],
          [ 1.8550,  1.8550,  1.8550,  ...,  1.7865,  1.5297,  1.3242],
          [ 1.8550,  1.8550,  1.8550,  ...,  1.6667,  1.1358,  0.6392],
          ...,
          [ 1.8550,  1.8550,  1.8550,  ...,  0.4337,  0.3309,  0.3481],
          [ 1.8550,  1.8550,  1.8550,  ...,  0.5707,  0.6049,  0.6392],
          [ 1.8550,  1.8550,  1.8550,  ...,  0.6392,  0.7419,  0.8104]],

         [[ 2.0259,  2.0259,  2.0259,  ...,  1.8859,  1.8683,  1.7983],
          [ 2.0259,  2.0259,  2.0259,  ...,  1.7983,  1.5532,  1.3256],
          [ 2.0259,  2.0259,  2.0259,  ...,  1.6232,  1.0455,  0.5378],
          ...,
          [ 2.0259,  2.0259,  2.0259,  ..., -0.2675, -0.3725, -0.3375],
          [ 2.0259,  2.0259,  2.0259,  ..., -0.1450, -0.1099, -0.0224],
          [ 2.0259,  2.0259,  2.0259,  ..., -0.0574,  0.0476,  0.1352]],

         [[ 2.2391,  2.2391,  2.2391,  ...,  2.1694,  2.1868,  2.1694],
          [ 2.2391,  2.2391,  

tensor([[[[-1.0562, -1.0562, -1.0390,  ..., -0.9705, -0.8507, -0.3541],
          [-1.0390, -1.0562, -1.0733,  ..., -0.9534, -1.0219, -0.9877],
          [-1.0390, -1.0562, -1.0733,  ..., -1.1247, -1.2617, -1.3302],
          ...,
          [-0.7993, -0.7822, -0.7479,  ...,  0.7933,  0.9303,  0.8447],
          [-0.7308, -0.7137, -0.6965,  ...,  0.5022,  0.3481,  0.0741],
          [-0.7137, -0.6965, -0.6623,  ..., -0.0972, -0.3712, -0.6452]],

         [[-0.7927, -0.7927, -0.7927,  ..., -0.8452, -0.7577, -0.2500],
          [-0.7927, -0.7927, -0.8102,  ..., -0.7577, -0.8803, -0.8627],
          [-0.7927, -0.7927, -0.8102,  ..., -0.9503, -1.1078, -1.1954],
          ...,
          [-0.5301, -0.5301, -0.5126,  ...,  0.9930,  1.0805,  0.9755],
          [-0.5126, -0.4951, -0.4601,  ...,  0.7129,  0.5378,  0.2227],
          [-0.4776, -0.4601, -0.4426,  ...,  0.1176, -0.1800, -0.4776]],

         [[-0.9504, -0.9504, -0.9504,  ...,  0.0256, -0.0790,  0.2696],
          [-0.9330, -0.9504, -

tensor([[[[-2.0494, -2.0494, -2.0837,  ...,  2.2489,  2.2489,  2.2489],
          [-2.0494, -2.0494, -2.0665,  ...,  2.2489,  2.2489,  2.2489],
          [-1.9980, -1.9980, -1.9809,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489]],

         [[-2.0357, -2.0357, -2.0357,  ...,  2.4286,  2.4286,  2.4286],
          [-2.0357, -2.0357, -2.0357,  ...,  2.4286,  2.4286,  2.4286],
          [-2.0357, -2.0357, -2.0182,  ...,  2.4286,  2.4286,  2.4286],
          ...,
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286]],

         [[-1.7522, -1.7522, -1.7522,  ...,  2.6400,  2.6400,  2.6400],
          [-1.7522, -1.7522, -

...

tensor([[[[-0.9877, -1.0733, -1.1760,  ..., -1.5357, -1.5357, -1.5185],
          [-1.1418, -1.2617, -1.3644,  ..., -1.4158, -1.3987, -1.4158],
          [-1.2788, -1.3987, -1.5014,  ..., -1.3302, -1.3302, -1.3473],
          ...,
          [ 1.7352,  1.8208,  1.7523,  ..., -0.6623, -0.6452, -0.6109],
          [ 1.4954,  1.5639,  1.5810,  ..., -0.6623, -0.5082, -0.3369],
          [ 1.0331,  1.1529,  1.3584,  ..., -0.4054, -0.1999, -0.0629]],

         [[-0.9153, -1.0203, -1.1078,  ..., -1.0903, -1.1078, -1.1078],
          [-1.1253, -1.2654, -1.3354,  ..., -0.9678, -0.9678, -0.9678],
          [-1.3004, -1.4230, -1.5455,  ..., -0.8277, -0.8102, -0.8277],
          ...,
          [ 1.9559,  2.0084,  1.9909,  ..., -0.4251, -0.3901, -0.3550],
          [ 1.6583,  1.7458,  1.8158,  ..., -0.4076, -0.2675, -0.1099],
          [ 1.2031,  1.3782,  1.5357,  ..., -0.1800,  0.0126,  0.1877]],

         [[-1.4733, -1.5081, -1.5256,  ..., -1.6999, -1.6824, -1.6476],
          [-1.3513, -1.4210, -

tensor([[[[-2.0323, -2.0323, -2.0323,  ..., -1.2103, -1.2103, -1.2103],
          [-2.0323, -2.0323, -2.0323,  ..., -1.2103, -1.2103, -1.2103],
          [-2.0323, -2.0323, -2.0323,  ..., -1.2103, -1.2103, -1.2103],
          ...,
          [-2.0323, -2.0323, -1.9467,  ..., -1.5185, -1.5014, -1.4672],
          [-2.0323, -2.0323, -2.0323,  ..., -1.5185, -1.5185, -1.5014],
          [-2.0323, -2.0323, -2.0323,  ..., -1.5528, -1.5357, -1.5185]],

         [[-1.9482, -1.9482, -1.9482,  ..., -0.4601, -0.4601, -0.4601],
          [-1.9482, -1.9482, -1.9482,  ..., -0.4601, -0.4601, -0.4601],
          [-1.9482, -1.9482, -1.9482,  ..., -0.4601, -0.4601, -0.4601],
          ...,
          [-1.9482, -1.9482, -1.7731,  ..., -1.0028, -0.9678, -0.9503],
          [-1.9482, -1.9482, -1.9482,  ..., -1.0203, -0.9853, -0.9678],
          [-1.9482, -1.9482, -1.9482,  ..., -1.0728, -1.0203, -1.0028]],

         [[-1.7173, -1.7173, -1.7173,  ..., -0.9853, -0.9853, -0.9853],
          [-1.7173, -1.7173, -

tensor([[[[-1.8268, -1.8610, -1.8782,  ...,  1.8550,  1.8379,  1.8208],
          [-1.8439, -1.8610, -1.9124,  ...,  1.8379,  1.8208,  1.7865],
          [-1.8610, -1.8782, -1.9295,  ...,  1.8208,  1.7865,  1.7694],
          ...,
          [-1.8782, -1.8782, -1.8953,  ...,  0.8104,  0.8276,  0.8618],
          [-1.8953, -1.8953, -1.8953,  ...,  0.8276,  0.8618,  0.8789],
          [-1.8953, -1.8953, -1.8953,  ...,  0.8618,  0.8789,  0.8789]],

         [[-1.4580, -1.4930, -1.5105,  ...,  2.0959,  2.0959,  2.0959],
          [-1.4755, -1.5105, -1.5280,  ...,  2.0959,  2.0784,  2.0784],
          [-1.5105, -1.5280, -1.5455,  ...,  2.1134,  2.0609,  2.0434],
          ...,
          [-1.6331, -1.6331, -1.6155,  ...,  1.7633,  1.7808,  1.7808],
          [-1.6506, -1.6331, -1.6155,  ...,  1.7633,  1.7983,  1.7983],
          [-1.6506, -1.6331, -1.6155,  ...,  1.7808,  1.8158,  1.8158]],

         [[-1.6127, -1.5953, -1.5779,  ...,  2.2391,  2.2391,  2.2217],
          [-1.6302, -1.6127, -

tensor([[[[-0.8335, -0.8335, -0.8335,  ..., -1.8953, -1.8953, -1.8953],
          [-0.8849, -0.8678, -0.8678,  ..., -0.4397, -0.4226, -0.4226],
          [-0.9363, -0.9363, -0.9363,  ..., -0.4054, -0.3883, -0.3883],
          ...,
          [-0.3369, -0.3198, -0.2856,  ..., -0.5938, -0.5082, -0.4054],
          [-0.3369, -0.2856, -0.2684,  ..., -0.5938, -0.5253, -0.4568],
          [-0.2856, -0.2684, -0.2342,  ..., -0.5253, -0.4739, -0.4054]],

         [[-0.6001, -0.6001, -0.6001,  ..., -1.8081, -1.8081, -1.8081],
          [-0.6527, -0.6352, -0.6352,  ..., -0.1099, -0.1099, -0.1099],
          [-0.6877, -0.6877, -0.6877,  ..., -0.0574, -0.0574, -0.0399],
          ...,
          [-0.0049,  0.0126,  0.0476,  ..., -0.2675, -0.1800, -0.0749],
          [ 0.0126,  0.0476,  0.0651,  ..., -0.2675, -0.2150, -0.1275],
          [ 0.0301,  0.0651,  0.1001,  ..., -0.1975, -0.1450, -0.0749]],

         [[-1.0724, -1.0898, -1.0898,  ..., -1.5779, -1.5779, -1.5779],
          [-1.1073, -1.1247, -


KeyboardInterrupt



## With Auto Deep Learning

In [42]:
import os
import logging
import pandas as pd
from typing import List, Dict


def create_df_image_folder(
    columns: List[str], dtypes: Dict[str, str]
) -> pd.DataFrame:
    """
    Create the dataframe that will be the base for the image folder dataset.
    
    Parameters
    ----------
    columns : list
        The columns that will be used for the dataframe
    
    dtypes : dict
        Dictionary for the column name : column dtype for the dataframe
    
    Returns
    -------
    df : pd.DataFrame
        The DataFrame that will be used for organizing the Image Folder information
    """

    df = pd.DataFrame(columns=columns)

    df = df.astype(dtypes)

    return df


def image_folder_convertion(
    parent_folder_path: str,
    columns: List[str] = [
        'image_path',
        'class',
        'split_type'
    ],
    dtypes: Dict[str, str] = {
        'image_path': 'object',
        'class': 'category',
        'split_type': 'category'
    }
):
    """
    Function to make the conversion from the image folder structure to a dataframe that is wanted.
    
    Parameters
    ----------
    parent_folder_path : str
        The folder for which we have the ['train', 'test'] folders.

    columns : list
        The columns that will be used for the dataframe
    
    dtypes : dict
        Dictionary for the column name : column dtype for the dataframe
    
    Returns
    -------
    df : pd.DataFrame
        The DataFrame that will be used for organizing the Image Folder information
    """

    # Get the df we will use as the base one 
    df = create_df_image_folder(
        columns=columns,
        dtypes=dtypes
    )

    # We get the folders, which we expect to be ['train', 'test', 'valid']
    split_folders: List[str] = os.listdir(parent_folder_path)
    
    for split_folder in split_folders:
        # We get the path of .../train and .../test
        split_path: str = parent_folder_path + '/' + split_folder

        # List the values that we have inside of each folder, which would be the classes
        children_classes: List[str] = os.listdir(split_path)

        # For each class of each folder
        for child_class in children_classes:
            # Get the path of .../train/class
            images_path: str = split_path + '/' + child_class

            # List the images we have inside that folder
            images_child_class_path: List[str] = os.listdir(images_path) 

            images_child_class_path = [images_path + image_path for image_path in images_child_class_path]
            class_list: List[str] = [child_class] * len(images_child_class_path)
            dtype_list: List[str] = [split_folder] * len(images_child_class_path)

            # TODO: This is hardtypping the columns, so not flexible (ImageFolder is not flexible neither)
            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        data = {
                            'class': class_list,
                            'split_type': dtype_list,
                            'image_path': images_child_class_path,
                    })
                ], axis=0
            )

            del images_path
            del images_child_class_path
            del class_list
            del dtype_list

    return df

In [44]:
df = image_folder_convertion('./raw_images_clean/')

In [45]:
df

,image_path,class,split_type
0,./raw_images_clean//test/ADONIS5.jpg,ADONIS,test
1,./raw_images_clean//test/ADONIS3.jpg,ADONIS,test
2,./raw_images_clean//test/ADONIS1.jpg,ADONIS,test
3,./raw_images_clean//test/ADONIS4.jpg,ADONIS,test
4,./raw_images_clean//test/ADONIS2.jpg,ADONIS,test
...,...,...,...
117,./raw_images_clean//train/CLEARWING MOTH055.jpg,CLEARWING MOTH,train
118,./raw_images_clean//train/CLEARWING MOTH016.jpg,CLEARWING MOTH,train
119,./raw_images_clean//train/CLEARWING MOTH091.jpg,CLEARWING MOTH,train
120,./raw_images_clean//train/CLEARWING MOTH103.jpg,CLEARWING MOTH,train


In [ ]:
import os
from typing import Optional, List, Dict

import torch
import numpy as np
import pandas as pd
from PIL import Image

from torch.utils.data import Dataset
import torchvision.transforms as transforms


class Creator(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        class_groups: List[str],
        transformation: transforms.Compose,
    ):

        self.df = df     
        self.transformation = transformation
        self.class_groups = class_groups

        # TODO: Get dummies for each of the classes


    @property
    def __len__(self) -> int:
        return len(self.df)
    

    @property
    def columns(self) -> List[str]:
        return self.df.columns.values.tolist()
    

    @property
    def class_groups(self) -> List[List[str]]:
        return self.class_groups
    

    @property
    def class_group_num(self) -> int:
        return len(self.class_groups)


    @property
    def class_group_unique(
        self,
        column: str
    ) -> List[str]:
    
        return self.df[column].unique().tolist()


    def __getitem__(
        self,
        idx
    ):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get which is the image path
        img_path = self.df.at[idx, 'image_path']
        image = Image.open(img_path)

        # TODO: Based on the class label, get which is the index based on the get_dummies
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)

        # TODO: Return as many classes as class that we have
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
    